In [1]:
import wfdb
import os
import pandas as pd
import numpy as np

In [2]:
root = 'training2017'

In [3]:
with open('training2017/RECORDS', 'r') as f:
    heads = f.readlines()
heads = [head.strip() for head in heads]

In [4]:
signals = {}
for head in heads:
    record = wfdb.rdrecord(os.path.join(root, head))
    signal = record.p_signal
    signal = np.array(signal).reshape(-1)
    signals[head] = signal

In [5]:
label_df = pd.read_csv('REFERENCE-v3.csv', header=None)
label_df.columns = ['head', 'label']
label_dict = {'N': 0, 'A': 1, 'O': 2, '~': 3}
label_df.replace(label_dict, inplace=True)

/var/folders/s2/gg0gc5y53z979n2bskjqp6kc0000gn/T/ipykernel_71529/3283022013.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  label_df.replace(label_dict, inplace=True)


In [6]:
data = {}
for k, v in signals.items():
    cnt = len(v) // 3000
    if cnt == 0:
        continue
    data[k] = [v[cnt*3000:(cnt+1)*3000] for cnt in range(cnt)]

In [7]:
df = pd.DataFrame.from_dict(data, orient='index').reset_index()
mf = df.melt(id_vars='index', value_name='signal').sort_values(by=['index', 'variable'])
mf.dropna(inplace=True)
exp = mf['signal'].apply(pd.Series)
ddf = pd.concat([mf.drop('signal', axis=1), exp], axis=1)
ddf.drop('variable', axis=1, inplace=True)
ddf.reset_index(drop=True, inplace=True)
ddf.columns = ['head'] + list(range(3000))
ddf

,head,0,1,2,3,4,5,6,7,8,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
0,A00001,-0.127,-0.162,-0.197,-0.229,-0.245,-0.254,-0.261,-0.265,-0.268,...,-0.058,-0.057,-0.055,-0.053,-0.051,-0.048,-0.045,-0.042,-0.039,-0.035
1,A00001,-0.030,-0.024,-0.020,-0.016,-0.014,-0.011,-0.007,-0.001,0.009,...,-0.061,-0.058,-0.054,-0.047,-0.034,-0.023,-0.013,-0.002,0.011,0.026
2,A00001,0.032,0.036,0.040,0.044,0.047,0.049,0.051,0.053,0.055,...,0.036,0.019,0.008,0.000,-0.007,-0.012,-0.015,-0.018,-0.022,-0.021
3,A00002,0.128,0.157,0.189,0.226,0.250,0.257,0.262,0.265,0.268,...,-0.071,-0.072,-0.074,-0.075,-0.077,-0.077,-0.077,-0.074,-0.070,-0.066
4,A00002,-0.063,-0.061,-0.059,-0.057,-0.054,-0.051,-0.049,-0.047,-0.045,...,0.543,0.591,0.625,0.639,0.648,0.658,0.673,0.700,0.708,0.699
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26935,A08527,0.069,0.076,0.088,0.114,0.139,0.157,0.173,0.189,0.205,...,-0.051,-0.074,-0.098,-0.123,-0.151,-0.166,-0.171,-0.173,-0.174,-0.173
26936,A08527,-0.169,-0.161,-0.150,-0.137,-0.130,-0.126,-0.122,-0.118,-0.116,...,-0.086,-0.080,-0.075,-0.071,-0.068,-0.065,-0.060,-0.054,-0.044,-0.036
26937,A08528,-0.237,-0.286,-0.338,-0.390,-0.425,-0.437,-0.443,-0.445,-0.442,...,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.010,-0.011,-0.013
26938,A08528,-0.014,-0.016,-0.017,-0.019,-0.021,-0.023,-0.024,-0.026,-0.028,...,0.240,0.371,0.504,0.592,0.597,0.535,0.360,0.131,-0.094,-0.269


In [8]:
final_df = ddf.merge(label_df, on='head')
final_df

,head,0,1,2,3,4,5,6,7,8,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,label
0,A00001,-0.127,-0.162,-0.197,-0.229,-0.245,-0.254,-0.261,-0.265,-0.268,...,-0.057,-0.055,-0.053,-0.051,-0.048,-0.045,-0.042,-0.039,-0.035,0
1,A00001,-0.030,-0.024,-0.020,-0.016,-0.014,-0.011,-0.007,-0.001,0.009,...,-0.058,-0.054,-0.047,-0.034,-0.023,-0.013,-0.002,0.011,0.026,0
2,A00001,0.032,0.036,0.040,0.044,0.047,0.049,0.051,0.053,0.055,...,0.019,0.008,0.000,-0.007,-0.012,-0.015,-0.018,-0.022,-0.021,0
3,A00002,0.128,0.157,0.189,0.226,0.250,0.257,0.262,0.265,0.268,...,-0.072,-0.074,-0.075,-0.077,-0.077,-0.077,-0.074,-0.070,-0.066,0
4,A00002,-0.063,-0.061,-0.059,-0.057,-0.054,-0.051,-0.049,-0.047,-0.045,...,0.591,0.625,0.639,0.648,0.658,0.673,0.700,0.708,0.699,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26935,A08527,0.069,0.076,0.088,0.114,0.139,0.157,0.173,0.189,0.205,...,-0.074,-0.098,-0.123,-0.151,-0.166,-0.171,-0.173,-0.174,-0.173,0
26936,A08527,-0.169,-0.161,-0.150,-0.137,-0.130,-0.126,-0.122,-0.118,-0.116,...,-0.080,-0.075,-0.071,-0.068,-0.065,-0.060,-0.054,-0.044,-0.036,0
26937,A08528,-0.237,-0.286,-0.338,-0.390,-0.425,-0.437,-0.443,-0.445,-0.442,...,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.010,-0.011,-0.013,0
26938,A08528,-0.014,-0.016,-0.017,-0.019,-0.021,-0.023,-0.024,-0.026,-0.028,...,0.371,0.504,0.592,0.597,0.535,0.360,0.131,-0.094,-0.269,0


In [9]:
len(final_df)

26940

In [10]:
int(final_df.iloc[0, -1])

0

In [11]:
from ECGDatasets import CINC2017Dataset
from utils import get_one_hot

In [12]:
mydataset = CINC2017Dataset(ecg_dir='training2017', ann_dir='REFERENCE-v3.csv',
                            target_transform=get_one_hot)

/Users/xiaoyudembp/Desktop/ecg/ECGDatasets.py:37: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  labels.replace(label_dict, inplace=True)


In [13]:
from torch.utils.data import DataLoader

In [14]:
mydataloder = DataLoader(mydataset, batch_size=32, shuffle=True)

In [15]:
ecg, label = next(iter(mydataloder))

tensor([[0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 0., 1., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [0., 0., 1., 0.]])